In [8]:
import pandas as pd

# Load the datasets with only the required columns
df1 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/final_animedataset.csv', usecols=['user_id', 'anime_id', 'my_score', 'score', 'scored_by', 'gender', 'popularity'])
df2 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/users-score-2023.csv', usecols=['user_id', 'anime_id', 'Anime Title', 'rating'])

# Merge the datasets on user_id and anime_id
final_df = pd.merge(df2, df1, on=['user_id', 'anime_id'])

# Display the first few rows to check the data
print(final_df.head())


   user_id  anime_id             Anime Title  rating  my_score gender  score  \
0        1        21               One Piece       9         9   Male   8.54   
1        1        48             .hack//Sign       7         7   Male   7.09   
2        1       320                  A Kite       5         5   Male   6.66   
3        1        49        Aa! Megami-sama!       8         8   Male   7.38   
4        1       304  Aa! Megami-sama! Movie       8         8   Male   7.63   

   scored_by  popularity  
0     423868          35  
1      61485         650  
2      18934        1946  
3      20930        1807  
4      18571        2007  


In [9]:
# Frequency encoding for categorical columns
categorical_columns = ['gender', 'Anime Title']

for col in categorical_columns:
    freq = final_df[col].value_counts()
    final_df[col] = final_df[col].map(freq)

# Handle missing values by filling with median for numerical columns
numerical_columns = ['my_score', 'score', 'scored_by', 'popularity']
for col in numerical_columns:
    final_df[col].fillna(final_df[col].median(), inplace=True)

# Display the first few rows after preprocessing
print(final_df.head())


C:\Users\Guest01\AppData\Local\Temp\ipykernel_16580\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)
C:\Users\Guest01\AppData\Local\Temp\ipykernel_16580\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

   user_id  anime_id  Anime Title  rating  my_score   gender  score  \
0        1        21        10462       9         9  4175028   8.54   
1        1        48         5341       7         7  4175028   7.09   
2        1       320         1892       5         5  4175028   6.66   
3        1        49         2448       8         8  4175028   7.38   
4        1       304         2395       8         8  4175028   7.63   

   scored_by  popularity  
0     423868          35  
1      61485         650  
2      18934        1946  
3      20930        1807  
4      18571        2007  


C:\Users\Guest01\AppData\Local\Temp\ipykernel_16580\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)
C:\Users\Guest01\AppData\Local\Temp\ipykernel_16580\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [10]:
# Define features (X) and target (y)
X = final_df.drop(columns=['rating'])  # All columns except 'rating' are features
y = final_df['rating']  # 'rating' is the target variable

# Display the shapes of X and y to confirm
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (6021419, 8), y shape: (6021419,)


In [11]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import time
import psutil
import numpy as np

# Function to calculate and return metrics for Random Forest with RandomizedSearchCV
def calculate_metrics(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor()

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'n_estimators': [30, 50],  # Number of trees in the forest
        'max_depth': [10, 20],  # Maximum depth of the tree
        'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2, 4, 8],  # Minimum number of samples required to be at a leaf node
        'max_features': ['sqrt', 'log2'],  # Number of features to consider when looking for the best split
        'bootstrap': [True, False]  # Whether to bootstrap samples when building trees
    }

    random_search = RandomizedSearchCV(
        rf, 
        param_distributions, 
        n_iter=10,  # Number of iterations for RandomizedSearchCV
        cv=3,  # Cross-validation folds
        scoring='neg_mean_squared_error',  # Scoring metric
        random_state=42,  # Ensures reproducibility
        n_jobs=-1  # Use all available cores
    )

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [13]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Metrics for sample size 1.0:
RMSE: 0.24719404113928103
MAPE: 0.009350085060578517
R2: 0.978013196404833
MSE: 0.06110489397476856
nRMSE: 0.027466004571031225
Execution Time (Raw): 4239.519551992416
Normalized Time (s/MB): 10.26407376342834
Average CPU Usage: 53.3
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 0.2723873715735306
MAPE: 0.010943056073591192
R2: 0.9732852839911382
MSE: 0.07419488019273664
nRMSE: 0.030265263508170066
Execution Time (Raw): 1653.2485933303833
Normalized Time (s/MB): 7.997178649982815
Average CPU Usage: 41.400000000000006
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 0.2762166901848925
MAPE: 0.01183251532573448
R2: 0.9725288741942455
MSE: 0.07629565993669689
nRMSE: 0.03069074335387694
Execution Time (Raw): 728.4655587673187
Normalized Time (s/MB): 7.047544056359279
Average CPU Usage: 40.8
Sample Size: 0.25
-------------------------------------------

In [14]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 116 minutes and 9.62 seconds
Total Normalized Execution Time for Entire Process: 8.92485153 seconds per MB
Total Average CPU Usage for Entire Process: 42.08%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.247194,0.009350,0.978013,0.061105,0.027466,4239.519552,10.264074,53.30,1.000
1,0.272387,0.010943,0.973285,0.074195,0.030265,1653.248593,7.997179,41.40,0.500
2,0.276217,0.011833,0.972529,0.076296,0.030691,728.465559,7.047544,40.80,0.250
3,0.277761,0.011998,0.972221,0.077151,0.030862,320.287168,6.197240,41.70,0.125
4,0.782780,0.108253,0.859462,0.612745,0.097848,0.318840,58.043099,41.45,100.000
5,0.377119,0.033375,0.947798,0.142219,0.041902,0.542679,9.879175,40.85,1000.000
6,0.371657,0.025634,0.953417,0.138129,0.041295,2.999210,5.459895,36.15,10000.000
7,0.283164,0.016050,0.971252,0.080182,0.031463,24.241150,4.412967,41.00,100000.000
